<a href="https://colab.research.google.com/github/RohanC07/Programming_for_DataScience/blob/main/Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This dataset represents the **All India Consumer Price Index (CPI)** for both rural and urban areas up to September 2014. The CPI measures changes in the average price level of a basket of consumer goods and services over time, serving as an important indicator of inflation in the country. The dataset covers various regions, helping to analyze price trends and inflationary pressures in rural and urban sectors. The data is crucial for understanding regional price disparities and evaluating the impact of inflation on different population segments.


**(1)** **Importing the CSV file into the Google Colab!**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**(2)** Importing necessary libraries and loading the CSV file using **Pandas,numpy,pyplot**,**seaborn** and **plotly express**

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import os
import glob
from scipy.stats import ttest_ind
from scipy import stats
import plotly.express as px

# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Assesment/Programming for Data Science/datafile.csv")
data
